# Automatic differentiation with `autograd`
Prior to `v0.4` PyTorch used the class `Variable` to record gradients. You had to wrap `Tensor`s in `Variable`s.
`Variable`s behaved exactly like `Tensors`.

With `v0.4` `Tensor` can record gradients directly if you tell it do do so, e.g. `torch.ones(3., requires_grad=True)`.
There is no need for `Variable` anymore.
Many tutorials still use `Variable`, be aware!

Simply add `requires_grad=True` to the tensors you want to calculate the gradients for. PyTorch libraries such as `nn.Module` track gradients automatically.

## Autograd is disabled by default

In [ ]:
import torch as pt

In [ ]:
x = pt.tensor(2.)
x

In [ ]:
y = pt.tensor(3.)
y

In [ ]:
z = x + y

In [ ]:
x.requires_grad, y.requires_grad, z.requires_grad,

### `backward()` is equivalent to `backward(tensor(1.0))`

In [ ]:
z.requires_grad = True

z.backward(pt.tensor(1.0)) 

z.grad, x.grad, y.grad

## Enable autograd for a tensor with `requires_grad = True`

In [ ]:
x = pt.tensor(2., requires_grad=True)
x

In [ ]:
x.requires_grad

In [ ]:
print(x.grad)

## Non-leaf tensors acquire autograd by default

In [ ]:
x = pt.tensor(2., requires_grad=True)
z = pt.tensor(3.)

#the product of x and z is a non-leaf tensor
x * z

x.requires_grad, z.requires_grad

In [ ]:
y = x ** 2

y.requires_grad, y.grad_fn

In [ ]:
y = x ** 2
y.backward()

x.grad, z.grad

## Ignore autograd with `no_grad()`; useful for inference

In [ ]:
x = pt.tensor(2., requires_grad=True)

with pt.no_grad():
    y = x * x
    print(y.grad_fn)

In [ ]:
from torch import nn

In [ ]:
model = nn.Linear(1, 1)
model.weight.data
model.bias.grad

<html><h2><font style="color:red">GPU support is required</font> to demonstrate high performance tensor operations</h2></html>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('In Google Colab select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")
print('using device:', device)  

When CUDA drivers are installed on the device, the following returns the CUDA capability profile documented by nVidia. For the capability profile v6.0 check out https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capability-6-x for information about the capability profile include the expected count of the Arithmetic Logic Units (ALUs).


In [ ]:
pt.cuda.get_device_capability(device) if pt.cuda.is_available() else None

## PyTorch tensor operations require tensors to reside on the same device 
* Use the `to` function to transfer a tensor to a specific device

In [ ]:
a = pt.ones(100).to("cpu")
b = pt.ones(100).to("cuda")
a + b

In [ ]:
import timeit

MAX_SIZE = 28
sizes = [2 ** i for i in range(MAX_SIZE)]

def benchmark_cpu_gpu(n):
  for device in ["cpu", "cuda"]:
    for size in sizes:
      a = pt.randn(size).to(device)
      b = pt.randn(size).to(device)
      yield timeit.timeit(lambda: a + b, number = n)

measurements = list(benchmark_cpu_gpu(1))

## Measure the performance ratio CPU / GPU, higher is faster GPU

In [ ]:
cpu = measurements[:MAX_SIZE]
gpu = measurements[MAX_SIZE:]
ratios = [cpu[i] / gpu[i] for i in range(len(cpu))]
ratios

## When adding tensors with `16,384` values and more, GPU can be `150x` faster

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.figure(figsize=(12,6))
plt.plot(sizes, ratios)
plt.xscale("log", basex=2)

## GPU performance on tensors with `8,192`  or fewer elements is on par with CPUs

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(sizes[:17], ratios[:17])
plt.xscale("log", basex=2);

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.